In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

import mlflow
from mlflow.data.pandas_dataset import PandasDataset
from mlflow.tracking import MlflowClient
from mlflow.models import ModelSignature, infer_signature

In [2]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [3]:
df = pd.read_csv('s3://mlops-personal-project/Traning-Data/202301-capitalbikeshare-tripdata.csv')


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


# EDA

In [59]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,65F0ACD101BF0D49,classic_bike,2023-01-04 19:34:07,2023-01-04 19:39:29,East Falls Church Metro / Sycamore St & 19th St N,31904.0,W Columbia St & N Washington St,32609.0,38.885321,-77.156427,38.885621,-77.166917,member
1,D75158CE73DC43F0,classic_bike,2023-01-27 15:26:38,2023-01-27 19:21:36,Carroll & Westmoreland Ave,32025.0,Fenton St & Ellsworth Dr,32036.0,38.975000,-77.011210,38.997033,-77.025608,member
2,33E85889625FF7CA,classic_bike,2023-01-05 20:44:38,2023-01-05 20:51:18,15th & L St NW,31276.0,Thomas Circle,31241.0,38.903649,-77.034918,38.905900,-77.032500,member
3,E1F055A1651F47A1,classic_bike,2023-01-03 17:45:14,2023-01-03 17:57:23,Hartland Rd & Harte Pl,32255.0,Merrifield Cinema & Merrifield Town Center,32235.0,38.878601,-77.222808,38.870093,-77.229970,member
4,88CC90CEEC298BAF,classic_bike,2023-01-03 05:18:46,2023-01-03 05:25:50,Merrifield Cinema & Merrifield Town Center,32235.0,Hartland Rd & Harte Pl,32255.0,38.870093,-77.229970,38.878601,-77.222808,member


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204077 entries, 0 to 204076
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             204077 non-null  object 
 1   rideable_type       204077 non-null  object 
 2   started_at          204077 non-null  object 
 3   ended_at            204077 non-null  object 
 4   start_station_name  195428 non-null  object 
 5   start_station_id    195428 non-null  float64
 6   end_station_name    194680 non-null  object 
 7   end_station_id      194680 non-null  float64
 8   start_lat           204077 non-null  float64
 9   start_lng           204077 non-null  float64
 10  end_lat             203856 non-null  float64
 11  end_lng             203856 non-null  float64
 12  member_casual       204077 non-null  object 
dtypes: float64(6), object(7)
memory usage: 20.2+ MB


In [8]:
df.describe()

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
count,195428.000000,194680.000000,204077.000000,204077.000000,203856.000000,203856.000000
mean,31371.335464,31372.851808,38.903504,-77.030524,38.902397,-77.030256
std,252.082794,250.491407,0.024856,0.031781,0.024574,0.031553
min,31000.000000,31000.000000,38.782633,-77.380000,38.780000,-77.380000
25%,31211.000000,31214.000000,38.893860,-77.042900,38.893028,-77.043074
50%,31278.000000,31277.000000,38.903165,-77.030000,38.902674,-77.030000
75%,31608.000000,31609.000000,38.913761,-77.012210,38.912644,-77.012108
max,32901.000000,32901.000000,39.130000,-76.820000,39.130000,-76.820000


In [17]:
print(df.value_counts(df['member_casual']),"\n")
print(df.value_counts(df['rideable_type']), '\n')

member_casual
member    141515
casual     62562
Name: count, dtype: int64 

rideable_type
classic_bike     171419
electric_bike     25814
docked_bike        6844
Name: count, dtype: int64 



In [27]:
categ_clumns = ['rideable_type', 'start_station_name', 'start_station_id', 'end_station_name','end_station_id', 'member_casual']

for i in categ_clumns:
    print(f"{i}: {df[i].nunique()}")

rideable_type: 3
start_station_name: 714
start_station_id: 712
end_station_name: 716
end_station_id: 714
member_casual: 2


# Preprocessing

In [4]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    
    All args must be of equal length.    
    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km


In [5]:
def preprocessing(df):
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    
    df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
    df = df[df['duration']>0]
        

    df['started_day'] = df['started_at'].dt.day
    df['started_hour'] = df['started_at'].dt.hour
    df['ended_day'] = df['ended_at'].dt.day
    df['ended_hour'] = df['ended_at'].dt.hour

    
    Q1 = df['duration'].quantile(0.25)
    Q3 = df['duration'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Remove outliers
    df_filtered = df[(df['duration'] >= lower_bound) & (df['duration'] <= upper_bound)]

    df['distance'] = haversine_np(df['start_lng'],df['start_lat'], df['end_lng'],df['end_lat'])
    
    categ_clumns = ['rideable_type', 'member_casual']
    df = pd.get_dummies(df, columns=categ_clumns, drop_first=True)

    deleted_columns=['started_at','ended_at','ride_id', 'start_lng', 'start_lat', 'end_lng', 'end_lat', 'start_station_name', 'start_station_id', 'end_station_name','end_station_id']
    df.drop(columns=deleted_columns, inplace=True)
    df.dropna(inplace=True)

    return df

In [6]:
processed_df = df.copy()
processed_df = preprocessing(processed_df)
processed_df.head()

/tmp/ipykernel_4252/4094010884.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['started_day'] = df['started_at'].dt.day
/tmp/ipykernel_4252/4094010884.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['started_hour'] = df['started_at'].dt.hour
/tmp/ipykernel_4252/4094010884.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,duration,started_day,started_hour,ended_day,ended_hour,distance,rideable_type_docked_bike,rideable_type_electric_bike,member_casual_member
0,5.366667,4,19,4,19,0.909569,False,False,True
1,234.966667,27,15,27,19,2.750975,False,False,True
2,6.666667,5,20,5,20,0.326598,False,False,True
3,12.150000,3,17,3,17,1.132374,False,False,True
4,7.066667,3,5,3,5,1.132374,False,False,True


In [7]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203840 entries, 0 to 204076
Data columns (total 9 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     203840 non-null  float64
 1   started_day                  203840 non-null  int32  
 2   started_hour                 203840 non-null  int32  
 3   ended_day                    203840 non-null  int32  
 4   ended_hour                   203840 non-null  int32  
 5   distance                     203840 non-null  float64
 6   rideable_type_docked_bike    203840 non-null  bool   
 7   rideable_type_electric_bike  203840 non-null  bool   
 8   member_casual_member         203840 non-null  bool   
dtypes: bool(3), float64(2), int32(4)
memory usage: 8.4 MB


In [23]:
processed_df.describe()

,duration,started_day,started_hour,ended_day,ended_hour,distance
count,203840.000000,203840.000000,203840.000000,203840.000000,203840.000000,203840.000000
mean,17.101671,15.610601,13.892450,15.612034,14.079636,1.793853
std,90.080579,9.059831,4.773341,9.060186,4.812777,1.375488
min,0.016667,1.000000,0.000000,1.000000,0.000000,0.000000
25%,6.100000,7.000000,10.000000,7.000000,10.000000,0.887611
50%,10.233333,16.000000,15.000000,16.000000,15.000000,1.486619
75%,17.050000,24.000000,17.000000,24.000000,18.000000,2.363547
max,23935.983333,31.000000,23.000000,31.000000,23.000000,22.653637


In [8]:
X = processed_df.drop('duration', axis=1)
y = processed_df['duration']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Setup MLflow

In [5]:
mlflow.set_tracking_uri(uri="http://ec2-16-171-38-100.eu-north-1.compute.amazonaws.com:5000")

In [11]:
mlflow.set_experiment(experiment_name="Bike-Rides")

<Experiment: artifact_location='mlflow-artifacts:/398162422483971143', creation_time=1709724198408, experiment_id='398162422483971143', last_update_time=1709724198408, lifecycle_stage='active', name='Bike-Rides', tags={}>

# Find the base model

In [12]:
# Signature
output_example = y_train[0]
print(output_example)

input_example = X_train.iloca[[0]]
input_example

signature = infer_signature(X_train, y_train)
signature

5.366666666666666


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


inputs: 
  ['started_day': integer (required), 'started_hour': integer (required), 'ended_day': integer (required), 'ended_hour': integer (required), 'distance': double (required), 'rideable_type_docked_bike': boolean (required), 'rideable_type_electric_bike': boolean (required), 'member_casual_member': boolean (required)]
outputs: 
  ['duration': double (required)]
params: 
  None

In [13]:
models = {
    # "Linear Regression": LinearRegression(),
    # "Ridge": Ridge(),
    # "Lasso": Lasso(),
    "Elastic Net": ElasticNet(),
    # "Decision Tree": DecisionTreeRegressor(),
    # "SVR": SVR(),
    # "Gradient Boosting": GradientBoostingRegressor(),
}

In [78]:
for name, model in models.items():

    with mlflow.start_run(run_name=''):

        mlflow.set_tag("Model_name", name)

        model.fit(X_train, y_train)

        predictions = model.predict(X_test)

        mse = mean_squared_error(y_test, predictions)
        
        
        if hasattr(model, 'alpha'):  # For Ridge, Lasso, ElasticNet
            mlflow.log_param("alpha", model.alpha)
        if hasattr(model, 'l1_ratio'):  # For ElasticNet
            mlflow.log_param("l1_ratio", model.l1_ratio)
        if hasattr(model, 'n_estimators'):  # For GradientBoosting
            mlflow.log_param("n_estimators", model.n_estimators)
        if hasattr(model, 'max_depth'):  # For Decision Tree and potentially others
            mlflow.log_param("max_depth", model.max_depth)
        if hasattr(model, 'C'):  # For SVR
            mlflow.log_param("C", model.C)
        if hasattr(model, 'kernel'):  # For SVR
            mlflow.log_param("kernel", model.kernel)
        if hasattr(model, 'min_samples_split'):  # For Decision Tree and GradientBoosting
            mlflow.log_param("min_samples_split", model.min_samples_split)
        if hasattr(model, 'min_samples_leaf'):  # For Decision Tree and GradientBoosting
            mlflow.log_param("min_samples_leaf", model.min_samples_leaf)
        if hasattr(model, 'learning_rate'):  # For GradientBoosting
            mlflow.log_param("learning_rate", model.learning_rate)

        
        # Log the model
        mlflow.sklearn.log_model(model, name, signature=signature)
        
        # Log metrics
        mlflow.log_metric("mse", mse)
        
        print(f"{name}: Model trained and logged with MSE: {mse}")


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Ridge: Model trained and logged with MSE: 2763.986211386135


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.025e+08, tolerance: 1.566e+05
  model = cd_fast.enet_coordinate_descent(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode

Lasso: Model trained and logged with MSE: 2152.6817577631437


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Elastic Net: Model trained and logged with MSE: 2089.731128072627


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


MlflowException: API request to http://ec2-16-171-38-100.eu-north-1.compute.amazonaws.com:5000/api/2.0/mlflow-artifacts/artifacts/398162422483971143/83d2c6544ef04db9aa68f7b7f9dd8020/artifacts/SVR/model.pkl failed with exception HTTPConnectionPool(host='ec2-16-171-38-100.eu-north-1.compute.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/398162422483971143/83d2c6544ef04db9aa68f7b7f9dd8020/artifacts/SVR/model.pkl (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))

## Trying the autologing

In [14]:
mlflow.autolog()

2024/03/09 16:50:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [15]:
for name, model in models.items():

    with mlflow.start_run(run_name='auto log'):

        mlflow.set_tag("Model_name", name)

        model.fit(X_train, y_train)

        predictions = model.predict(X_test)

        mse = mean_squared_error(y_test, predictions)
        
        print(f"{name}: Model trained and logged with MSE: {mse}")


2024/03/09 16:52:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2024/03/09 16:52:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64

Elastic Net: Model trained and logged with MSE: 1593.7505467257913


# Hyperparameter Tuning The Best Model

In [19]:
model = ElasticNet(max_iter=10000)
param_grid = {'alpha': [0.01,0.1, 1.0], 'l1_ratio': [0.1, 0.5, 0.9]}

In [20]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')

In [15]:
MLflowDataset = mlflow.data.from_pandas(
    processed_df, name='Processed data', targets='duration'
)

/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


In [21]:
with mlflow.start_run():
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)

    # Log the best parameters and MSE metric
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("mse", mse)

    # Log the model
    mlflow.sklearn.log_model(best_model, "ElasticNet")

    mlflow.set_tag("Model_name", "ElasticNet")
    mlflow.log_input(MLflowDataset, context="training")
    

    print("Best Parameters: ", grid_search.best_params_)
    print("Test MSE: ", mse)


/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.120e+06, tolerance: 3.020e+04
  model = cd_fast.enet_coordinate_descent(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.381e+05, tolerance: 3.020e+04
  model = cd_fast.enet_coordinate_descent(
/home/mohammed/.local/share/virtualenvs/MLOps_Project-ode336_0/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might

Best Parameters:  {'alpha': 0.01, 'l1_ratio': 0.9}
Test MSE:  13203.117402265598


## Trying the autolog

In [ ]:
mlflow.autolog()


# MLflow Client

In [7]:
client = MlflowClient()

In [15]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/810246490971712463', creation_time=1710095872450, experiment_id='810246490971712463', last_update_time=1710095872450, lifecycle_stage='active', name='Bike-Rides Duration Prediction: 2023-01', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/398162422483971143', creation_time=1709724198408, experiment_id='398162422483971143', last_update_time=1710094955621, lifecycle_stage='active', name='Bike-Rides-test', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1709536084320, experiment_id='0', last_update_time=1709536084320, lifecycle_stage='active', name='Default', tags={}>]

In [25]:
registered_models = client.get_registered_model('Best Model: 2023-01')

In [38]:
runs = client.search_runs(
    experiment_ids=398162422483971143,
    max_results=5,
    order_by=["metrics.mse ASC"]
)

runs

[<Run: data=<RunData: metrics={'mse': 1846.6443418120907}, params={'C': '1.0'}, tags={'Model_name': 'SVR',
  'mlflow.log-model.history': '[{"run_id": "9569ec9303f24a94a3e7b197e1c236cc", '
                              '"artifact_path": "SVR", "utc_time_created": '
                              '"2024-03-06 18:11:10.833129", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.10.12", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.4.1.post1", '
                              '"serialization_format": "cloudpickle", "code": '
                              'null}}, "model_uuid": '
                

## Just testing

In [23]:
experiment_id = client.get_experiment_by_name(f"Bike-Rides Duration Prediction: 2023-01").experiment_id
runs = client.search_runs(experiment_id, order_by=["metrics.training_score DESC"], max_results=5)
best_run_id = runs[0].info.run_id
# print(best_run_id)
best_run_details = client.get_run(best_run_id)
# print(best_run_details.data)
print(best_run_details.data.metrics['training_mean_squared_error'])
print(best_run_details.data.tags['estimator_name'])

best_run_artifact_uri = best_run_details.info.artifact_uri

print(best_run_artifact_uri)

2619.259309047345
GradientBoostingRegressor
mlflow-artifacts:/810246490971712463/9b5a1541b23d4ed28d1ade701251f1d0/artifacts


# Interacting with Model Registry

In [50]:
run_id=runs[0].info.run_id
run_id

'9569ec9303f24a94a3e7b197e1c236cc'

In [51]:
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="predict_bike_rides_duration")

Registered model 'predict_bike_rides_duration' already exists. Creating a new version of this model...
2024/03/08 17:06:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: predict_bike_rides_duration, version 2
Created version '2' of model 'predict_bike_rides_duration'.


<ModelVersion: aliases=[], creation_timestamp=1709906785445, current_stage='None', description='', last_updated_timestamp=1709906785445, name='predict_bike_rides_duration', run_id='9569ec9303f24a94a3e7b197e1c236cc', run_link='', source='mlflow-artifacts:/398162422483971143/9569ec9303f24a94a3e7b197e1c236cc/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [52]:
model_name = "predict_bike_rides_duration"
latedt_ver=client.get_latest_versions(model_name)

for ver in latedt_ver:
    print(f"version: {ver.version}, stage: {ver.current_stage}")

/tmp/ipykernel_999/2397937747.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.0/model-registry.html#migrating-from-stages
  latedt_ver=client.get_latest_versions(model_name)


version: 2, stage: None
version: 1, stage: Staging


In [53]:
client.transition_model_version_stage(
    name = model_name,
    version=2,
    stage= "Production",
    archive_existing_versions= True
)

/tmp/ipykernel_999/3217734253.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1709906785445, current_stage='Production', description='', last_updated_timestamp=1709906988972, name='predict_bike_rides_duration', run_id='9569ec9303f24a94a3e7b197e1c236cc', run_link='', source='mlflow-artifacts:/398162422483971143/9569ec9303f24a94a3e7b197e1c236cc/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>